<a href="https://colab.research.google.com/github/SHAMIK-97/PhysiscsFormer/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"
!pip install numpy
!pip install cityscapesscripts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.1/953.1 kB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.1/450.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 33.1 MB/s eta 0:00:00


In [ ]:
!pip install ftfy

In [ ]:
import torch
device = "cpu"
if torch.cuda.is_available:
  print('All good, a Gpu is available')
  device = torch.device("cuda:0")

else:
  print('Please set GPU via Edit -> Notebook Settings.')

In [ ]:
from cityscapesscripts.download import downloader
from typing import Any, Callable, Dict, List, Optional, Union, Tuple
from torchvision.datasets import Cityscapes
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import os

COLAB_DIR = '/content/data/'

if not os.path.exists(COLAB_DIR):
  !mkdir /content/data
# registration on https://www.cityscapes-dataset.com/
session = downloader.login()
downloader.get_available_packages(session=session)

# data for semantic segmentation task
print('Downloading gtFine and leftImg8bit packages ...\n')
package_list =['gtFine_trainvaltest.zip','leftImg8bit_trainvaltest.zip']
downloader.download_packages(session=session, package_names=package_list, destination_path=COLAB_DIR)

!unzip -q /content/data/leftImg8bit_trainvaltest.zip -d /content/data/
!unzip -q /content/data/gtFine_trainvaltest.zip -d /content/data/

In [ ]:
from cityscapesscripts.helpers import labels
def encode_segmap(mask):

  mask_copy = mask.copy()
  for label in labels.labels:
    mask_copy[mask==label.id] = label.trainId
    mask_copy[mask_copy==255] = 19
  return mask_copy

# transform=A.Compose(
# [
#     A.Resize(128, 128),
#     A.HorizontalFlip(),
#     #A.Normalize(mean=(0.385, 0.356, 0.306), std=(0.329, 0.324, 0.325)),
#     ToTensorV2(),
# ]
# )

class MyClass(Cityscapes):
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        image = Image.open(self.images[index]).convert('RGB')
        image = np.array(image)


        targets: Any = []
        for i, t in enumerate(self.target_type):
            if t == 'polygon':
                target = self._load_json(self.targets[index][i])
            else:
                target = Image.open(self.targets[index][i])
                target = np.array(target)
                target = encode_segmap(target)
            targets.append(target)
        target = tuple(targets) if len(targets) > 1 else targets[0]

        if self.transforms is not None:
            pass #transformed=transform(image=image, mask=target)
        return image,target

In [ ]:
from PIL import Image
data_train= MyClass('/content/data/', split='train', mode='fine',
                     target_type='semantic') #,transforms=transform)

data_val = MyClass('/content/data/', split='val', mode='fine',
                     target_type='semantic') #,transforms=transform)

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
 # Loader
num_workers = os.cpu_count()
size_batch_train = 5
size_batch_val = 5 #2 * size_batch_train

loader_train = torch.utils.data.DataLoader(data_train, batch_size=size_batch_train,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=num_workers)
loader_val = torch.utils.data.DataLoader(data_val, batch_size=size_batch_val,
                                         shuffle=False,
                                         num_workers=num_workers)

In [ ]:
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!git checkout main
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 16289, done.
remote: Total 16289 (delta 0), reused 0 (delta 0), pack-reused 16289
Receiving objects: 100% (16289/16289), 23.19 MiB | 10.39 MiB/s, done.
Resolving deltas: 100% (11373/11373), done.
/content/mmsegmentation
Already on 'main'
Your branch is up to date with 'origin/main'.
Obtaining file:///content/mmsegmentation
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation


In [ ]:
import mmseg
from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv
import cv2
import glob
import os
import time
from os.path import join, isdir
from os import listdir, rmdir
from shutil import move, rmtree, make_archive
import pickle
print(mmseg.__version__)
# Example output: 0.24.1

1.2.1


In [ ]:
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torchsummary import summary
import torch.nn as nn
import numpy as np
import os
from typing import Callable, Dict, List, Tuple, Union
from torch import nn, optim, utils
import torch.nn.functional as F
from itertools import permutations

torch.cuda.empty_cache()

def _get_relu(name:str)-> nn.Sequential:

  container = nn.Sequential()
  relu = nn.ReLU()
  container.add_module(f'{name}_relu', relu)

  return container

def _max_pool2D(name:str)-> nn.Sequential:

  container = nn.Sequential()
  pool1 = nn.MaxPool2d(kernel_size=(3,3), stride=1)
  container.add_module(f'{name}_maxpool_2d', pool1)
  container.add_module(f'{name}_maxpool_2d_pad_1', nn.ConstantPad2d(1,1))
  return container



class PhysicsFormer(nn.Module):
  def __init__(self, model_initializer, num_classes,inference=False) -> None:
    super(PhysicsFormer,self).__init__()
    self.model_initializer = model_initializer
    self.num_classes = num_classes
    self.relu = _get_relu('relu')
    self.maxpool_1 = _max_pool2D("maxpool_1")
    self.pad = nn.ConstantPad2d(1,1)
    self.inference = inference

  def opening(self, x):

    relu = x
    x1 =  self.maxpool_1(x)
    x2 =  torch.matmul(x1, relu)
    x3 =  self.maxpool_1(x2)
    x4 =  torch.matmul(x3, relu)
    # x5 =  self.maxpool_1(x4)
    # x6 =  torch.matmul(x5, relu)
    # x7 =  self.maxpool_1(x6)
    # x8 =  torch.matmul(x7, relu)
    return x4

  def forward(self, input):
     output = inference_model(self.model_initializer, input)
     logits = output.seg_logits.data
     if self.inference:
      return output.pred_sem_seg.data
     li = []
     for i in range(self.num_classes):
        li.append(logits[i:i+1,::])


     perm = []
     for i in permutations(li, 2):
        perm.append(i)

     final_tensors = []
     for i in perm:
       concatenated_tensor = torch.cat((i[0],i[1]), dim=0)
       concat_softmax = concatenated_tensor.softmax(dim=0)
       concat_diff = concat_softmax.diff(dim=0)
       concat_relu = self.relu(concat_diff)
       final_tensors.append(concat_relu)
     final_concatenated = torch.cat(final_tensors, dim=0)
     final_concatenated = self.pad(final_concatenated)
     final_concatenated_closed = torch.mul(final_concatenated,-1)
     opened_tensor = self.opening(final_concatenated_closed)
     closed_tensor = self.opening(final_concatenated)
     multiplied_tensor = torch.matmul(opened_tensor, final_concatenated_closed)
     multiplied_tensor_closed = torch.matmul(closed_tensor, final_concatenated)
     normalized_tensor = F.normalize(multiplied_tensor)
     normalized_tensor_closed = F.normalize(multiplied_tensor_closed)
     tensor_sub = torch.sub(final_concatenated_closed,normalized_tensor,alpha=1)
     tensor_sub_closed = torch.sub(final_concatenated,normalized_tensor_closed,alpha=1)
     norm_closed =  torch.norm(tensor_sub_closed,p=1)
     norm = torch.norm(tensor_sub,p=1)
     final_norm = torch.abs(norm_closed - norm)
     return final_norm,logits

In [ ]:
from tqdm import tqdm
#import torchmetrics
def train(model: nn.Module,
          train_loader: DataLoader,
          optimizer: torch.optim,
          criterion: Callable[[torch.Tensor, torch.Tensor], float]) -> float: #Tuple[float, float]: #
    """Train loop to train a neural network for one epoch.

    Args:
        model: the model to train.
        train_loader: the data loader containing the training data.
        device: the device to use to train the model.
        optimizer: the optimizer to use to train the model.
        criterion: the loss to optimize.
        epoch: the number of the current epoch

    Returns:
        the L1 Loss value on the training data,
        the accuracy on the training data.
    """

    samples_train = 0
    loss_train = 0
    correct = 0
    train_ce_loss = 0
    #metric_train = 0
    #size_ds_train = len(train_loader.dataset)


    eps = 1e-11
    L1_values = []
    # IMPORTANT: from now on, since we will introduce batch norm, we have to tell PyTorch if we are training or evaluating our model
    model.train()
    for idx_batch, (image, labels) in tqdm(enumerate(train_loader), total=2975, desc='Sample'): #595

      #labels =  torch.tensor(labels).to(device)

      optimizer.zero_grad()
      final_norm,scores = model(image)
      ce_loss = criterion(scores, labels.long())

      final_loss = ce_loss + eps*final_norm

      loss_train += final_loss.item()*len(image)
      train_ce_loss += ce_loss.item()*len(image)
      samples_train += len(image)
      L1_values.append(final_norm)

      final_loss.backward()
      optimizer.step()
      #correct += iou_stats(scores, labels)


    loss_train /= samples_train
    train_ce_loss /= samples_train
    #accuracy = 100. * correct / samples_train
    print(f'loss--->{loss_train}')
    #print(f'accuracy--->{accuracy}')

    return loss_train, L1_values,train_ce_loss #,accuracy

In [ ]:
def training_loop(num_epochs: int,
                  optimizer: torch.optim,
                  scheduler: torch.optim,
                  model: nn.Module,
                  loader_train: DataLoader)->Dict:

    """Executes the training loop.

        Args:
            name_exp: the name for the experiment.
            num_epochs: the number of epochs.
            optimizer: the optimizer to use.
            model: the mode to train.
            loader_train: the data loader containing the training data.
            loader_val: the data loader containing the validation data.


        Returns:
            A dictionary with the statistics computed during the train:
            the values for the train loss for each epoch
            the values for the train accuracy for each epoch

        """
    criterion =  nn.CrossEntropyLoss(ignore_index=19)



    losses_values = []
    train_acc_values = []
    l1_values = []
    train_ce = []
    #val_acc_values = []
    for epoch in range(1, num_epochs+1):
        print(f'Epoch----{epoch}/{num_epochs}')
        loss_train,L1_values,train_ce_loss = train(model, loader_train,optimizer,criterion)   #accuracy_train
        #loss_val, accuracy_val = validate(model, loader_val, device, criterion)

        losses_values.append(loss_train)
        l1_values.append(L1_values)
        train_ce.append(train_ce_loss)
        #train_acc_values.append(accuracy)
        #val_acc_values.append(accuracy_val)
        scheduler.step()
        #lr =  optimizer.param_groups[0]['lr']


    return {'loss_values': losses_values,
            'L1_values': l1_values,
            'train_ce_loss': train_ce}
            #'train_acc_values': train_acc_values}
            #'val_acc_values': val_acc_values}

In [ ]:
!mim download mmsegmentation --config segformer_mit-b4_8xb1-160k_cityscapes-1024x1024 --dest .

processing segformer_mit-b4_8xb1-160k_cityscapes-1024x1024...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.4/234.4 MiB 66.3 MB/s eta 0:00:00
Successfully downloaded segformer_mit-b4_8x1_1024x1024_160k_cityscapes_20211207_080709-07f6c333.pth to /content/mmsegmentation
Successfully dumped segformer_mit-b4_8xb1-160k_cityscapes-1024x1024.py to /content/mmsegmentation


In [ ]:
config_file = 'segformer_mit-b4_8xb1-160k_cityscapes-1024x1024.py'
checkpoint_file = '/content/mmsegmentation/segformer_mit-b4_8x1_1024x1024_160k_cityscapes_20211207_080709-07f6c333.pth'

import gc


gc.collect()

lr =  0.00006
weight_decay=0.001
num_epochs = 15
num_classes = 19


# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cpu')
#result = inference_model(model, img)
physicsFormer = PhysicsFormer(model,num_classes)

if os.path.exists('/content/data/physicsFormer.pth'):
  physicsFormer.load_state_dict(torch.load('/content/data/physicsFormer.pth'))
  print(f'******model_loaded***********')


#*** IMPORTANT: remember to create the optimizer AFTER you have moved the model to the GPU, the tensors storing the parameter are not the same used by the original model ***
optimizer = torch.optim.AdamW(physicsFormer.parameters(), lr=lr,weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer,total_iters=13, power=1.0, last_epoch=- 1, verbose=False)


statistics = training_loop(num_epochs, optimizer,scheduler,physicsFormer, data_train)

loss_value = statistics['loss_values']
acc_value =  statistics['train_acc_values']
L1_values =  statistics['L1_values']
train_ce_loss = statistics['train_ce_loss']


print(f'loss value: {loss_value}')
print(f'accuracy values: {acc_value}')
print(f'L1 loss values: {L1_values}')
print(f'Cross_entropy loss values: {train_ce_loss}')

Loads checkpoint by local backend from path: /content/mmsegmentation/segformer_mit-b4_8x1_1024x1024_160k_cityscapes_20211207_080709-07f6c333.pth
Epoch----1/15


Sample:   0%|          | 0/2975 [00:32<?, ?it/s]


RuntimeError: ignored

In [ ]:
model = init_model(config_file, checkpoint_file, device='cuda:0')

/content/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:249: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: /content/mmsegmentation/segformer_mit-b4_8x1_1024x1024_160k_cityscapes_20211207_080709-07f6c333.pth


In [ ]:
from cityscapesscripts.helpers import labels
from PIL import Image

video = '/content/data/leftImg8bit/val'
for filename in os.listdir(video):
  for files in os.listdir(video+'/'+filename):
    image_read = mmcv.imread(video+'/'+filename+'/'+files)
    result = inference_model(model, image_read)
    result = result.pred_sem_seg.data.squeeze()
    result = result.cpu().numpy()
    results_copy = result.copy()
    color_result = np.zeros([1024,2048,3])
    for label in labels.labels:
      results_copy[result==label.trainId] = label.id
      color_result[result==label.trainId] = label.color
    results_copy = results_copy.astype(dtype = np.uint8)
    color_result = color_result.astype(dtype = np.uint8)
    im = Image.fromarray(results_copy)
    im_color = Image.fromarray(color_result)
    if not os.path.exists('/content/out-folder'):
      os.mkdir('/content/out-folder/')
    if not os.path.exists('/content/color/out-folder'):
      os.mkdir('/content/color/')
      os.mkdir('/content/color/out-folder/')
    im.save('/content/out-folder/'+files.replace('_leftImg8bit','*'))
    im_color.save('/content/color/out-folder/'+files.replace('_leftImg8bit','*'))


In [ ]:
import os
import subprocess
import sys

os.environ['CITYSCAPES_DATASET'] = '/content/data/'
os.environ['CITYSCAPES_RESULTS'] = '/content/out-folder/'



In [ ]:
!python /usr/local/lib/python3.10/dist-packages/cityscapesscripts/evaluation/evalPixelLevelSemanticLabeling.py

Evaluating 500 pairs of images...
Images Processed: 500 

--------------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------- 
              |   u   |   e   |   r   |   o   |   s   |   d   |   g   |   r   |   s   |   p   |   r   |   b   |   w   |   f   |   g   |   b   |   t   |   p   |   p   |   t   |   t   |   v   |   t   |   s   |   p   |   r   |   c   |   t   |   b   |   c   |   t   |   t   |   m   |   b   |  Prior |
--------------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------

In [ ]:
result.seg_logits.data.shape

torch.Size([19, 1024, 2048])

In [ ]:
result.pred_sem_seg.data.shape

torch.Size([1, 1024, 2048])

In [ ]:
import sys
import os
#sys.path.append('/scratch_net/biwirender09/shbasu/conda_envs/shbasuenv/lib/python3.7/site-packages/')
try:
    from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
except:
    print('execute pip install transformers')
try:
    import numpy as np
except:
    print('execute pip install numpy')
try:
    from cityscapesscripts.helpers import labels
except:
    print('execute pip install cityscapesscripts')

from PIL import Image
import os
import requests
from torch import nn



feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b5-finetuned-cityscapes-1024-1024")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b5-finetuned-cityscapes-1024-1024")

path = '/content/inference/data/leftImg/leftImg8bit/val/' #'/scratch_net/biwirender09/shbasu/leftImg8bit_trainvaltest/leftImg8bit/val/'
for folders in os.listdir(path):
    for files in os.listdir(path+folders):
        image = Image.open(path+folders+'/'+files)
        inputs = feature_extractor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
        logits = nn.functional.interpolate(logits.detach().cpu(), size=(1024,2048), mode="bilinear", align_corners=False)
        logits = logits.argmax(dim=1)
        result = logits.numpy()
        result = result.reshape(1024,2048)
        results_copy = result.copy()
        #color_result = np.zeros([1024,2048,3])
        #trainId_image = np.zeros([1024,2048])
        for label in labels.labels:
            results_copy[result==label.trainId] = label.id
            #color_result[result==label.trainId] = label.color
            #trainId_image[result==label.trainId] = label.trainId
        results_copy = results_copy.astype(dtype = np.uint8)
        #color_result = color_result.astype(dtype = np.uint8)
        #trainId_image = trainId_image.astype(dtype = np.uint8)
        im = Image.fromarray(results_copy)  #.resize((1024,2048), resample=Image.BILINEAR)
        #im_color = Image.fromarray(color_result).resize((1024,2048), resample=Image.BILINEAR)
        #im_trainId = Image.fromarray(trainId_image).resize((1024,2048), resample=Image.BILINEAR)
        if not os.path.exists('/content/inference/out-folder'):
          os.mkdir('/content/inference/out-folder/')
        if not os.path.exists('/content/inference/out-folder/labelIds/'):
          os.mkdir('/content/inference/out-folder/labelIds')
        # if not os.path.exists('/content/inference/out-folder/color_maps/'):
        #   os.mkdir('/content/inference/out-folder/color_maps/')
        # if not os.path.exists('/content/inference/out-folder/train_id/'):
        #   os.mkdir('/content/inference/out-folder/train_id/')
        im.save('/content/inference/out-folder/labelIds/'+files.replace('_leftImg8bit','*'))
        #im_color.save('/content/inference/out-folder/color_maps/'+files.replace('_leftImg8bit','_color'))
        #im_trainId.save('/content/inference/out-folder/train_id/'+files.replace('_leftImg8bit','_trainId'))

        # im.save('/scratch_net/biwirender09/shbasu/results/labelIds/'+files.replace('_leftImg8bit','*'))
        # im_color.save('/scratch_net/biwirender09/shbasu/results/color_maps/'+files.replace('_leftImg8bit','_color'))
        # im_trainId.save('/scratch_net/biwirender09/shbasu/results/train_id/'+files.replace('_leftImg8bit','_trainId'))

In [ ]:
!zip -r /content/out-folder_1  /content/color/out-folder

  adding: content/color/out-folder/ (stored 0%)
  adding: content/color/out-folder/munster_000061_000019*.png (deflated 13%)
  adding: content/color/out-folder/frankfurt_000001_049770*.png (deflated 6%)
  adding: content/color/out-folder/frankfurt_000000_009969*.png (deflated 10%)
  adding: content/color/out-folder/frankfurt_000001_075984*.png (deflated 9%)
  adding: content/color/out-folder/frankfurt_000000_015389*.png (deflated 13%)
  adding: content/color/out-folder/frankfurt_000001_023369*.png (deflated 12%)
  adding: content/color/out-folder/munster_000173_000019*.png (deflated 12%)
  adding: content/color/out-folder/frankfurt_000001_007857*.png (deflated 13%)
  adding: content/color/out-folder/munster_000135_000019*.png (deflated 19%)
  adding: content/color/out-folder/munster_000000_000019*.png (deflated 11%)
  adding: content/color/out-folder/lindau_000013_000019*.png (deflated 13%)
  adding: content/color/out-folder/munster_000107_000019*.png (deflated 15%)
  adding: content/c

In [ ]:
!zip -r /content/out-folder_2  /content/out-folder

  adding: content/out-folder/ (stored 0%)
  adding: content/out-folder/munster_000061_000019*.png (deflated 5%)
  adding: content/out-folder/frankfurt_000001_049770*.png (deflated 2%)
  adding: content/out-folder/frankfurt_000000_009969*.png (deflated 3%)
  adding: content/out-folder/frankfurt_000001_075984*.png (deflated 3%)
  adding: content/out-folder/frankfurt_000000_015389*.png (deflated 6%)
  adding: content/out-folder/frankfurt_000001_023369*.png (deflated 5%)
  adding: content/out-folder/munster_000173_000019*.png (deflated 3%)
  adding: content/out-folder/frankfurt_000001_007857*.png (deflated 7%)
  adding: content/out-folder/munster_000135_000019*.png (deflated 11%)
  adding: content/out-folder/munster_000000_000019*.png (deflated 5%)
  adding: content/out-folder/lindau_000013_000019*.png (deflated 4%)
  adding: content/out-folder/munster_000107_000019*.png (deflated 6%)
  adding: content/out-folder/frankfurt_000000_022254*.png (deflated 9%)
  adding: content/out-folder/frank

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/inference/out-folder/ /content/drive/MyDrive/val_images/